In [86]:
%aiida
import aiida
import os
import numpy as np
from ase import units
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm import Group, JobCalculation,  WorkCalculation

In [2]:
work_group = "Al6xxxDB_structures_calc"
work_group = "twoatom_threerandom1_calc1"
work_group = "Al6xxxDB_passingsubset"
qb = QueryBuilder()
qb.append(Group, filters={'name': work_group}, tag='g')
qb.append(WorkCalculation, tag='job', member_of='g')
all_works = [x[0] for x in qb.all()]  # all workchains

In [111]:
def get_workcalc_runnerdata(worknode):
    ase_structure = worknode.inp.structure.get_ase()
    energy = worknode.out.output_parameters.get_attrs()['energy'] # units?
    forces = worknode.out.output_array.get_array('forces') # units?
    path = worknode.out.CALL.out.retrieved.get_abs_path()
    return ase_structure, energy, forces, worknode.uuid,path

In [117]:
angstrom_to_bohrradius = 1.8897261  
eV_to_Hartree = 0.036749325
eV_per_angstrom_to_hartree_per_bohrradius = 0.019446905


fileOut = open("input.data", "w")

for idx,workchain in enumerate(all_works[:2]):
    print(workchain.out)
    ase_structure, energy, forces,uuid,path = get_workcalc_runnerdata(workchain)
    #print('u',uuid,energy,path)
    forces=forces[0]

    fileOut.write("begin\ncomment\n")
    
    # write the cell
    cell=ase_structure.cell*angstrom_to_bohrradius   
    for idx_cell,i in enumerate(cell):
        fileOut.write("lattice %.10f %.10f %.10f\n" % (cell[idx_cell][0], cell[idx_cell][1],cell[idx_cell][2]))
    
    
    # write the positions
    nr_of_atoms = ase_structure.positions.shape[0]
    for idx_pos in range(nr_of_atoms):
        atCor = ase_structure.positions[idx_pos]*angstrom_to_bohrradius 
        atFor = forces[idx_pos]*eV_per_angstrom_to_hartree_per_bohrradius
        element = ase_structure.get_chemical_symbols()[idx_pos]
        fileOut.write("atom   %.6f    %.6f   %.6f %s  0.0   0.0  %.10f  %.10f  %.10f\n"  % (atCor[0], atCor[1],atCor[2],element, atFor[0],atFor[1],atFor[2]) )
    fileOut.write("energy %.15f\n" % (energy*eV_to_Hartree))
    fileOut.write("charge 0\nend\n")
    #print(ase_structure.cell)
    #print(ase_structure.positions)
    
fileOut.close() 

In [110]:
work.out.CALL.out.retrieved.get_abs_path()

NameError: name 'work' is not defined

In [90]:
1/units.Hartree*units.Bohr

0.01944690395740863

In [25]:
ase_structure.cell[0]

array([5.58476271, 0.        , 0.        ])

In [53]:
ase_structure.positions[1]

array([0.17113221, 2.90317995, 2.70237466])

In [7]:
ase_structure, energy, forces = get_workcalc_runnerdata(all_works[1])

In [8]:
energy

-3975.3160999561683

In [10]:
len(all_works)

50

In [12]:
ase_structure.positions

array([[0.        , 0.        , 0.        ],
       [0.17113221, 2.90317995, 2.70237466],
       [3.01201477, 0.20927424, 2.76638023],
       [3.11563402, 2.99687395, 5.5021047 ],
       [0.08125886, 5.56789519, 2.65663893],
       [0.08704806, 2.88724748, 5.56667914],
       [2.89024104, 0.13971072, 0.01353424],
       [2.82144483, 2.94484683, 2.94486589]])

In [13]:
ase_structure.cell

array([[5.58476271, 0.        , 0.        ],
       [0.        , 5.58476271, 0.        ],
       [0.        , 0.        , 5.58476271]])

In [59]:
forces[0][0]

array([ 0.00695175, -0.01126246,  0.00869521])

In [78]:
ase_structure.get_chemical_symbols

<bound method Atoms.get_chemical_symbols of Atoms(symbols='Al4Mg4', pbc=[True, True, True], cell=[5.584762710165715, 5.584762710165715, 5.584762710165715], masses=..., positions=...)>

In [79]:
ase_structure.get_chemical_symbols()

['Al', 'Al', 'Al', 'Al', 'Mg', 'Mg', 'Mg', 'Mg']